In [1]:
import sys, os  
import pandas as pd  
import numpy as np 
import cv2  

In [2]:
from tensorflow.keras.models import Sequential  
from keras.models import model_from_json  
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten  
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D, Input  
from tensorflow.keras.losses import categorical_crossentropy  
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import l2  
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from matplotlib import pyplot as plt
from PIL import Image

In [3]:
df=pd.read_csv("fer2013.csv")

In [4]:
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [5]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 30
width, height = 48, 48

In [6]:
X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

In [7]:
train_y=to_categorical(train_y, num_classes=num_labels)
test_y=to_categorical(test_y, num_classes=num_labels)

In [8]:
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)
X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [9]:
model = Sequential()
model.add(Input(shape=X_train.shape[1:]))  # Define the input shape here
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

In [10]:
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

In [11]:
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Flatten())

In [12]:
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_labels, activation='softmax'))
# model.summary()

In [ ]:
gen = ImageDataGenerator()
train_generator = gen.flow(X_train, train_y, batch_size=batch_size)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',  # Or you can use keras.optimizers.Adam()
    metrics=['accuracy']
)

model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs
)

Epoch 1/30


c:\Users\jeffr\OneDrive\Desktop\CS\HackUMass-2024\.conda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


448/448 ━━━━━━━━━━━━━━━━━━━━ 44s 92ms/step - accuracy: 0.2599 - loss: 1.7958
Epoch 2/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 93us/step - accuracy: 0.4062 - loss: 1.5992 
Epoch 3/30
  1/448 ━━━━━━━━━━━━━━━━━━━━ 1:07 151ms/step - accuracy: 0.3125 - loss: 1.5854

c:\Users\jeffr\OneDrive\Desktop\CS\HackUMass-2024\.conda\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


445/448 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.3791 - loss: 1.5826

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=SGD(),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Epoch 1/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 94s 200ms/step - accuracy: 0.5927 - loss: 1.0477 - val_accuracy: 0.5500 - val_loss: 1.1819
Epoch 2/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 90s 201ms/step - accuracy: 0.6002 - loss: 1.0337 - val_accuracy: 0.5628 - val_loss: 1.1873
Epoch 3/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 89s 197ms/step - accuracy: 0.6165 - loss: 1.0077 - val_accuracy: 0.5631 - val_loss: 1.1943
Epoch 4/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 88s 197ms/step - accuracy: 0.6278 - loss: 0.9840 - val_accuracy: 0.5756 - val_loss: 1.1613
Epoch 5/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 88s 197ms/step - accuracy: 0.6288 - loss: 0.9737 - val_accuracy: 0.5665 - val_loss: 1.1682
Epoch 6/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 89s 198ms/step - accuracy: 0.6380 - loss: 0.9536 - val_accuracy: 0.5701 - val_loss: 1.1560
Epoch 7/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 89s 197ms/step - accuracy: 0.6322 - loss: 0.9480 - val_accuracy: 0.5578 - val_loss: 1.1991
Epoch 8/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 89s 197ms/step - accuracy: 0.6457 - loss: 0

In [ ]:
fer_json = model.to_json()
with open("fer2.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer2.weights.h5")

In [ ]:
train_score = model.evaluate(X_train, train_y, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', 100*train_score[1])
test_score = model.evaluate(X_test, test_y, verbose=0)
print('Test loss:', test_score[0])
print('Test accuracy:', 100*test_score[1])

Train loss: 0.4124991297721863
Train accuracy: 87.76341676712036
Test loss: 1.3139702081680298
Test accuracy: 57.75982141494751


In [ ]:
def emotion_analysis(emotions):
  objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
  y_pos = np.arange(len(objects))
  plt.bar(y_pos, emotions, align='center', alpha=0.5)
  plt.xticks(y_pos, objects)
  plt.ylabel('percentage')
  plt.title('emotion')
  plt.show()

In [ ]:
#load model  
model = model_from_json(open("fer2.json", "r").read())  
#load weights  
model.load_weights('fer2.weights.h5') 

In [ ]:
face_haar_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")  
if face_haar_cascade.empty():
    print("Error: Haar Cascade file not loaded properly!")

In [ ]:
cap=cv2.VideoCapture(0)  
while True:  
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image  
    #if not ret:  
        #continue  
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)  

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)  


    for (x,y,w,h) in faces_detected:  
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)  
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image  
        roi_gray=cv2.resize(roi_gray,(48,48))  
        img_pixels = image.img_to_array(roi_gray)  
        img_pixels = np.expand_dims(img_pixels, axis = 0)  
        img_pixels /= 255  

        predictions = model.predict(img_pixels)  

        #find max indexed array  
        max_index = np.argmax(predictions[0])  

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')  
        predicted_emotion = emotions[max_index]
        
        if predicted_emotion is not "happy":
            predicted_emotion = "not happy"  

        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)  

    resized_img = cv2.resize(test_img, (1000, 700))  
    cv2.imshow('Facial emotion analysis ',resized_img)  



    if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed  
        break  

cap.release()  
cv2.destroyAllWindows  

<>:27: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:27: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\jeffr\AppData\Local\Temp\ipykernel_24892\3406053380.py:27: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if predicted_emotion is not "happy":


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━

KeyboardInterrupt: 

: 